# Isotonic Regression

### 1. 데이터 만들기

In [1]:
import pandas as pd
import numpy as np 
import torch 

In [2]:
# 계수의 차원 :5 , 관측치개수 10000개
p=5
n=10000

# x 관측치
X_ = torch.normal(10,1,size=(n,p),dtype=torch.float64)
X = torch.cat([torch.ones(n,1,dtype=torch.float64),X_],dim=1)

# 추정해야 될 beta 생성
real_beta = torch.tensor(range(p+1),dtype=torch.float64)

# y = X*(Real Beta) + E 로 y를 생성해줌
y = torch.mv(X,real_beta)+torch.randn(n,dtype=torch.float64)

### 2. 함수 생성

(1) $X$ matrix 를 다르게 변환할 $Z$ matrix, (2) $\beta$ 를 $\delta$ 로 재매개변수화 시켜줄 $A$ matrix, (3) $\delta$ 를 만들어줄 함수 생성 <br>
(4) Isotonic regression에서 lse를 minimize하는 $\delta$ 를 찾고 $\beta$ 를 구하는 함수 생성

In [3]:
def Zmatrix(x,p):
    value = torch.zeros(n,p+1,dtype = torch.float64)
    value[:,0]=1

    for i in range(p):
        value[:,(i+1)] = torch.sum(x[:,i+1:p+1],1)
    return value

def Amatrix(p):
    value = torch.zeros(p+1,p+1,dtype=torch.float64)
    value[0,0]=1
    value[1,1]=1
    
    for i in range(p-1):
        value[i+2][i+1] = -1.
        value[i+2][i+2] = 1.
    return value
    
def delta(b,p):
    value = torch.mv(Amatrix(p),b)
    return value

In [4]:
def Isotonic(X,y,mu,e,t,d,p):
    #Z matrix, delta 초기치 생성 
    A = Amatrix(p)
    Z = Zmatrix(X,p)
    # delta 초기치
    d = delta(beta,p)
    ##  Centering step
    for i in range(10000):
        # diag matrix들 생성 
        dr2 = torch.diag(1/(d**2))
        dr2[0:2,:]=0
        dr1 = torch.diag(1/d)
        dr1[0:2,:]=0
        # KKT 조건에 의해 nu에 대한 해 계산
        nu1 = torch.inverse(2*t*torch.mm(Z.T,Z) + dr2) 
        nu2 = 2*t*torch.mv(Z.T,y) - 2*t*torch.mv(torch.mm(Z.T,Z),d) + torch.mv(dr1,torch.ones(p+1,dtype=torch.float64))
        nu = torch.mv(nu1,nu2)
        # delta update
        d = d + nu
        # stopping criterion
        if ((p-1)/t)<e:
            break
        t = mu*t
        # beta 생성
    beta_new = torch.mv(torch.inverse(A),d)
    return beta_new

### 3.초기치 생성, 값 출력

In [5]:
e = 0.000001
mu = 1.1
t = 0.1
beta = torch.abs(torch.randn(p+1,dtype=torch.float64))
beta,_=torch.sort(beta)
beta.requires_grad=True

In [6]:
# beta 초기치
print(beta)

tensor([0.0795, 0.1473, 0.3992, 1.0460, 1.1357, 1.4807], dtype=torch.float64,
       requires_grad=True)


In [7]:
Isotonic(X,y,mu,e,t,beta,p)

tensor([0.0162, 1.0144, 2.0049, 2.9864, 4.0014, 4.9903], dtype=torch.float64,
       grad_fn=<MvBackward0>)

In [8]:
print(real_beta)

tensor([0., 1., 2., 3., 4., 5.], dtype=torch.float64)


- 최적화를 통해 구한 계수들이 추정할 beta 값에 근접한 값을 가짐을 보여주고 있다.